# Funksjoner

Vi starter så godt som hver oppgave med å laste ned et datasett fra GitHub-mappen til Ekte Data. Hva om vi hadde en funksjon som gjorde dette for oss, så vi slapp å bruke np.genfromtxt, sortere dataene inn i tid og dataverdier, og sette fyll-verdier til NaN gang på gang? Dette lager vi i denne oppgaven.

Å lage funksjoner er veldig praktisk når man har en operasjon man har bruk for ofte. Det er lurt å lage funsjonene sine generelle og ikke for omfattende - hvis de blir for spesifikke vil de bare fungere i helt spesielle tilfeller, og man får bruk for dem sjelden. 

In [ ]:
import numpy as np # for regneoperasjoner
import matplotlib.pyplot as plt # for figurer
import matplotlib

### Funksjon #1

Denne funksjonen skal kun lese inn et datasett fra githubmappen til Ekte Data og gi to output: tidsvektoren og selve dataene. 

Fra tidligere vet vi at de månedlige og de årlige datasettene må leses inn litt forskjellig: i det årlige trenger vi litt ekstra info om datasettet, nemlig at kolonnene for tid og variabel-veridene deles av komma, og at det er en linje med header øverst. Vi vet hva slags datasett vi skal laste inn, så det kan vi oppgi til funksjonen ved å sette `filetype='Y'` eller `filetype='M'`. Funksjonen starter derfor med en if-løkke som sjekker `filetype`: er det `'Y'` inkluderer den `delimeter` og `skip_header`, hvis `filetype` er `'M'` utelukker den `delimeter` og `skip_header`. 

In [ ]:
def loadData(file,filetype):
    
    '''
    Denne funksjonen skal
    - Lese inn et datasett fra github mappen til Ekte Data.
    - Returnere en tidsvektor og en matrise med selve verdiene.

    Input 
    file     : Navnet på .txt-filen (ikke inkluder stien her, kun filnavnet)
    filetype : 'M' hvis datasettet har månedlig oppløsning og 'Y' hvis det har 
    årlig oppløsning.
    
    Output
    data      : En vektor eller matrise (avhengig av om datasettet som leses inn
    er årlig eller månedlige gjennomsnitt) med alle verdiene i datasettet
    tid     : Tidsvektoren til datasettet.
    
    '''
    
    # Last inn dataene med np.genfromtxt som vi har brukt før
    if filetype == 'Y':
        data = np.genfromtxt(
            'https://raw.githubusercontent.com/irendundas/EkteData/main/'+file, 
            dtype=float, delimiter=',',skip_header=1)
    
    elif filetype == 'M':
        data = np.genfromtxt(
            'https://raw.githubusercontent.com/irendundas/EkteData/main/'+file, 
            dtype=float)
        
    # Sett alle "fyll-verdier" til NaN
    data[data==-999.99]=np.nan     
    
    # Tidsvektor med desimaler
    tid=data[:,0]
    
    # Selve dataene
    data=data[:,1:]
    
    return data, tid
    
    

Nå kan vi bruke funksjonen vår:

In [ ]:
data,tid = loadData(file='TempBergenYearly.txt',filetype='Y')

In [ ]:
data

In [ ]:
tid

### Oppgave 1 
Bruk funksjonen til å laste inn datasettet TempBergen.txt og skriv ut de to variablene data og tid. 

### Funksjon #2

Vi kan gjøre denne funksjonen litt enklere å bruke ved å utelukke `filetype` som input. Da må funksjonen selv sjekke om den trenger å inkludere `delimeter` og `skip_header`. Funksjonen trenger da et kriterie den kan sjekke som forteller den om det er en månedlig eller årlig fil den skal lese inn. Dette kan (som alt annet) sannsynligvis gjøres på mange ulige måter, men et sted å starte for å finne et passende kriterie er å se hva som skjer hvis vi prøver å laste inn et årlig datasett *uten* `delimeter` og `skip_header`:

In [ ]:
data=np.genfromtxt(
    'https://raw.githubusercontent.com/irendundas/EkteData/main/'+
    'TempBergenYearly.txt',dtype=float)
data

Alt blir NaN. Dette er et fint kriterie å bruke: dersom **alt** blir nan må man laste inn på metoden for årlig data. Det er viktig å presisere at kriteriet må være at **alt** er NaN - det kan jo være at datasettet inneholder et par NaN uten at det betyr at det er lastet inn feil. 

In [ ]:
# Alle verdier på disse indeksene er NaN
np.nonzero(data)

Men er **alle** indeksene i "data" NaN? 
Vi sjekker det ved å sjekke om lengden av vektoren `np.nonzero(data)` er like lang som "data"-vektoren. 

In [ ]:
print(len(data))

# Under må vi ha med [0] for å kutte vekk dtype=int64 etc.
print(len(np.nonzero(data)[0]))

Vi sjekker det også med en if-løkke.

In [ ]:
if len(data) == len(np.nonzero(data)[0]):
    print('Alle verdier i data er NaN')

Vi kan nå oppdatere funsjonen vår slik at vi slipper å oppgi om det er månedlig eller årlig oppløsning på datasettet vårt!

In [ ]:
def loadData(file):
    
    '''
    Denne funksjonen skal
    - Lese inn et datasett fra github mappen til Ekte Data.
    - Returnere en tidsvektor og en matrise med selve verdiene.

    Input 
    file     : Navnet på .txt-filen (ikke inkluder stien her, kun filnavnet)
    filetype : 'M' hvis datasettet har månedlig oppløsning og 'Y' hvis det har 
    årlig oppløsning.
    
    Output
    data      : En vektor eller matrise (avhengig av om datasettet som leses inn
    er årlig eller månedlige gjennomsnitt) med alle verdiene i datasettet
    tid     : Tidsvektoren til datasettet.
    
    '''
    
    # Last inn dataene med np.genfromtxt som vi har brukt før
    # Prøv først å laste det inn uten å opggi info om delimeter og skip_header
    data = np.genfromtxt(
            'https://raw.githubusercontent.com/irendundas/EkteData/main/'+file, 
            dtype=float)
    
    # Sjekk om dette funket eller om alt ble NaN.  Dersom alt ble NaN laster vi
    # inn datasettet på nytt og inkluderer info om delimeter og skip_header
    if len(data) == len(np.nonzero(data)[0]):
        data = np.genfromtxt(
            'https://raw.githubusercontent.com/irendundas/EkteData/main/'+file, 
            dtype=float, delimiter=',',skip_header=1)
        
    # Sett alle "fyll-verdier" til NaN
    data[data==-999.99]=np.nan     
    
    # Tidsvektor med desimaler
    tid=data[:,0]
    
    # Selve dataene
    data=data[:,1:]
    
    return data, tid
    

In [ ]:
data,tid = loadData(file='TempBergen.txt')

In [ ]:
data

In [ ]:
tid

### Oppgave 2

Ta utgangspunkt i funksjon #2 men utvid den slik at den også returnerer en gjennomsnittsverdi "dataMean". For årsdatasettet skal den returnere det totale gjennomsnittet over alle årene (én verdi), mens for månedsdatasettet skal den returnere gjennomsnittet for hver måned (12 verdier)


### Behandling av output

To ting om output:
- Selv om vi skriver `return data, tid`, kan vi kalle disse variablene hva vi vil når vi bruker funksjonen. Vet vi f.eks at vi laster inn et datasett over temperatur kan vi skrive `temp, tid = loadData(...)` istedet for `data, tid = loadData(...)`. Da skjønner funksjonen at `temp` er variabelnavnet på outputen `data`. 
- Vi trenger "hente ut" alle return-verdiene. Er vi kun interessert i tidsvektoren kan vi skrive `_,tid = loadData(...)`. Hvis vi ikke vet nøyaktig hva funksjonen inneholder kan vi også skrive `data = loadData()`: da kommer alle outputverdiene ut i rekkefølgen spesifisert av `return`.

### Oppgave 3

Funksjonen fra oppgave 2 gir tre output: data, tid og dataMean. 
- Skriv ut kun `data`-variabelen, men kall den `temp`. 
- Skriv ut alle variablene i en variabel som du kaller `vars`. Plukk så ut `temp`, `tid`, og `meanTemp` fra `vars`.

### Oppgave 4

I introen nevnte vi at det er lurt å lage funksjonene sine generelle. Å inkludere å ta gjennomsnittet inni `loadData` gjør den litt mindre generell. Det kan være ryddig å kun la `loadData` laste inn data, og istedet lage en ny funksjon som tar gjennomsnittet av disse dataene. 

Lag en ny funksjon som gjør to ting: 
- først skal den bruke funksjonen `loadData` til å hente inn datafilen, 
- så skal den ta gjennomsnittet på samme måte som i oppgave 2. 

Input variabelen blir `file`, og output variabelen blir kun `dataMean`.

Denne funksjonen skal altså bruke en annen funksjon inni seg, og så gi et nytt output. Et hint for å komme igang er at funksjonen `loadData` gjør jo også dette: både np.genfromtxt og np.nonzero er funksjoner som gir output som brukes videre i `loadData`.

________________

### Prosjektoppgave: Funksjon for gjennomsnitt i seksjoner

I flere av disse konsept-oppgavene er en del av oppgaven å dele tidsperioden inn i seksjoner som en måte å filtrere rådataene på. Dette kan vi lage en funksjon for slik at det gjøres automatisk. Vi tar utgangspunkt i funksjonene vi har over, og inkludere metoder for å finne indekser som deler inn datasettet i for eksempel tiårs-bolker som vi har gjort i f.eks. "for-løkke" oppgaven. 

Ting å tenke på:
- Hvilke ekstra input trengs?
- Hvilke output vil vi ha? 
- Hva skal funksjonen gjøre dersom antall år i hver bolk ikke går opp i antall år i datasettet?
- Skal den behandle "rest-årene" likt uavhengig av hvor mange det er? Hvis man f.eks. vil dele inn i tiårs-bolker - skal funksjonen gjøre det samme uavhengig av om det er 2 eller 9 restår?

### Løsnings*forslag*
Husk igjen - denne prosjektoppgaven kan løses på 

som i oppg 4 kan gjøre dette ryddigere og penere. 

Denne funksjonen skal
- lese inn et datasett fra github mappen til Ekte Data
- dele datasettet inn i bolker med lengde angitt av dT (antall år)
- kunne håndtere både datasettene med års- og måneds-gjennomsnitt
- putte "rest år" i en siste bolk dersom lengden ikke går opp med dT, og skrive ut en beskjed om hvilke år som er i denne bolken (dersom aktuelt). Dersom det er færre enn dT/2 restår - legg dem til den siste bolken, dersom det er dT/2 eller flere restår - lag en ny liten bolk. 

In [ ]:
def array_mean(
    file,dT, 
    filepath='https://raw.githubusercontent.com/irendundas/EkteData/main/'
):
    
    '''
    Denne funksjonen skal
    - lese inn et datasett fra github mappen til Ekte Data.
    - dele datasettet inn i bolker med lengde angitt av dT (antall år).
    - kunne håndtere både datasettene med års- og måneds-gjennomsnitt.
    - putte "rest år" i en siste bolk dersom lengden ikke går opp med dT, 
      og skrive ut en beskjed om hvilke år som er i denne bolken (dersom 
      aktuelt).
    - jeg vil gjerne at den skal kunne sjekke om den trenger å ta vekk header 
      osv selv, men per nå må man ha input på det.
    
    Input 
    file     : the name of the txt-file, not including the path, which is 
    constant and set below. 
    dT       : the timestep, i.e., the number of years to average across. 
    
    Output
    meanVal  : gjennomsnittsverdier i bolker med lengde bestemt av dT.
    time     : en tidsvektor med midtpunktene av hver tidsbolk.
    
    '''
    
    
    # Last inn dataene med np.genfromtxt som vi har brukt før
    # Prøv først å laste det inn uten å opggi info om delimeter og skip_header
    data = np.genfromtxt(filepath+file, dtype=float)
    
    # Sjekk om dette funket eller om alt ble NaN.  Dersom alt ble NaN laster vi
    # inn datasettet på nytt og inkluderer info om delimeter og skip_header
    if len(data) == len(np.nonzero(data)[0]):
        data = np.genfromtxt(filepath+file, 
                             dtype=float, delimiter=',',skip_header=1)
        
        
    data[data==-999.99]=np.nan
    T=len(data) # Antall år i datasettet
    
    start=0 # startindeks
    stop=T  # stopindeks
    
    # Start- og slutt-indeks til hver av "dT"-års-bolkene
    ind=np.arange(start,stop,dT) 
    
    
    # I tilegg til å returnere gjennomsnittsverdiene vil vi ha en vektor med 
    # tid. Dette gir midtpunktene i hver bolk med desimaler.
    time=np.arange(data[0,0]+dT/2,2021,10)
    

    # In this case dT doesn't add up into T, and some years will be left out. 
    # We want to take the mean over these as well, and include them at the end 
    # of the averaged time series. 
    if T-ind[-1] > 0: 
        rest=T-ind[-1] # nr of years left out
        if rest < dT/2:
            ind[-1]=T
            print((
                f'Dividing the datset into sections of length {dT} leaves out '
                f'{rest} year(s) at the end of the time series. The last ' 
                f'element in the output is the average of these/this {rest} '
                f'year(s) combined with the last full period.'
            ))
        elif rest >= dT/2:
            ind=np.append(ind,len(data)) 
            # Må få med en ekstra tidsperiode i de tilfellene hvor jeg legger 
            # til en ekstra bolk
            time=np.append(time,time[-1]+rest/2)
            print((
                f'Dividing the datset into sections of length {dT} leaves out '
                f'{rest} year(s) at the end of the time series. The last '
                f'element in the output is the average of these/this {rest} '
                f'year(s).'
            ))
    
    
    # dT-bolker x 1 eller 12 avhengig av om det skal bli års- eller månedssnitt
    meanVal=np.zeros((len(ind)-1,data.shape[1]-1)) 
    
    if data.shape[1]==2:
        for i in range(len(ind)-1): 
            # for e.g., i=0 blir dette data[ind[0]:ind[1],1]=data[0:dT,1]
            meanVal[i]=np.nanmean(data[int(ind[i]):int(ind[i+1]),1]) 
            
    elif data.shape[1]==13:
        for mo in range(12): # iterer gjennom månedene
            for i in range(len(ind)-1): # iterer gjennom dT-årsperiodene
                # vi begynner på "data" sin kolonne mo+1 fordi første kolonne i
                # "data" er årstallene 
                meanVal[i,mo]=np.nanmean(data[int(ind[i]):int(ind[i+1]),mo+1]) 
                
    # vi vil også gi ut en tidsvektor i heltall avrundet fra "time"-vektoren   
    #time=np.arange(start+dT/2,stop-dT/2,dT)
    meanTime = np.round(np.nextafter(time, time+1))
    #timeid = timeid.astype(int)
    #meanTime=data[timeid,0]
    
    return meanVal, time, meanTime

In [ ]:
array_mean(file='TempBergenYearly.txt',dT=13)